In [ ]:
kajal = True
try:
    module("unload", "cuda/11.6")
    module("load", "cuda/11.4")
    module("load","ffmpeg")
    module("list")
except: 
    kajal = False

In [ ]:
import sys
from pathlib import Path
if kajal:
    path_root = Path( '/cobra/u/kkumari/warp-drive')
    sys.path.append(str(path_root))

else:
    path_root = Path( '/project_ghent/MARL/warp-drive/')
    sys.path.append(str(path_root))
    path_root = Path( '/project_ghent/MARL/collective_MARL/')
    sys.path.append(str(path_root))
print(sys.path)

In [ ]:
# Log in to your W&B account
import wandb
"90dc2cefddde123eaac0caae90161981ed969abe"
wandb.login()

In [ ]:
import torch
# assert torch.cuda.device_count() > 0, "This notebook needs a GPU to run!"

from importlib import reload

import logging
# Set logger level e.g., DEBUG, INFO, WARNING, ERROR
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Load the run config.
from config import run_config

In [ ]:
from warp_drive.utils.env_registrar import EnvironmentRegistrar
from custom_env import CUDACustomEnv

env_registrar = EnvironmentRegistrar()
env_registrar.add_cuda_env_src_path(CUDACustomEnv.name, "custom_env_step_numba", env_backend="numba")

from warp_drive.env_wrapper import EnvWrapper
from warp_drive.training.utils.device_child_process.child_process_base import event_messenger

env_wrapper = EnvWrapper(
    env_obj=CUDACustomEnv(**run_config["env"]),
    num_envs=run_config["trainer"]["num_envs"],
    env_backend="numba",
    env_registrar=env_registrar,
    event_messenger=event_messenger,
    process_id=0
)

from warp_drive.training.models.factory import ModelFactory

model_factory = ModelFactory.add("predator_policy", "predator_policy", "PredatorPolicy") 
model_factory = ModelFactory.add("prey_policy", "prey_policy", "PreyPolicy") 


In [ ]:
policy_tag_to_agent_id_map = {
    "predator": list(env_wrapper.env.predators),
    "prey": list(env_wrapper.env.preys),
}

In [ ]:
from warp_drive.training.trainer_wandb import Trainer

trainer = Trainer(
    env_wrapper=env_wrapper,
    config=run_config,
    policy_tag_to_agent_id_map=policy_tag_to_agent_id_map,
    device_id=0,
    num_devices=1,
    verbose=False
)

### Train

In [ ]:
trainer.train()

### Load
#### From local

In [ ]:
import os

name = run_config["saving"]["name"]
tag = run_config["saving"]["tag"]

directory = f"/tmp/{name}/{tag}"
files = os.listdir(directory)
max_number = max([int(number) for number in files])

total_lenght = run_config["env"]["episode_length"]*run_config["trainer"]["num_episodes"]

trainer.load_model_checkpoint(
    {
        "prey": f"/tmp/{name}/{tag}/{max_number}/prey_{total_lenght}.state_dict",
        "predator": f"/tmp/{name}/{tag}/{max_number}/predator_{total_lenght}.state_dict",
    }
)

#### from wandb

In [ ]:
# Alternatively load from wandb
run = wandb.init(project="rl_project", entity=None, job_type='inference')
total_lenght = run_config["env"]["episode_length"]*run_config["trainer"]["num_episodes"]

for name in run_config["policy"]:
    print(name)
    artifact = run.use_artifact(f'tcazalet_airo/rl_project/{name}:latest', type='model')
    artifact_dir = artifact.download(root=f"/tmp/{name}_state_dict")
wandb.finish()


In [ ]:
trainer.load_model_checkpoint(
    {
        "prey": f"/tmp/prey_state_dict/prey_{total_lenght}.state_dict",
        "predator": f"/tmp/predator_state_dict/predator_{total_lenght}.state_dict",
    }
)

In [ ]:
import animations
reload(animations)
from animations import generate_animation
from IPython.display import HTML

predator_size = run_config["env"]["predator_size"]
prey_size = run_config["env"]["prey_size"]

# Visualize the entire episode roll-out
anim = generate_animation(trainer, predator_size=predator_size, prey_size=prey_size)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
# Close the trainer to clear up the CUDA memory heap
trainer.graceful_close()